In [11]:
import ee
import pandas as pd
from tqdm import tqdm

ee.Initialize(project='pranavkoka123') 

geometry = ee.Geometry.Rectangle([78.97, 30.70, 79.20, 30.79])

In [9]:
def calculate_ndsi_and_confidence(image):
    green = image.select('SR_B3').rename('Green')
    swir = image.select('SR_B6').rename('SWIR')
    ndsi = green.subtract(swir).divide(green.add(swir)).rename('NDSI')
    qa_pixel = image.select('QA_PIXEL')
    snow_confidence = qa_pixel.rightShift(5).bitwiseAnd(1).rename('Snow_Confidence')
    cloud_confidence = qa_pixel.rightShift(3).bitwiseAnd(1).rename('Cloud_Confidence')

    return image.addBands([ndsi, green, swir, snow_confidence, cloud_confidence])

In [4]:
def calculate_pixel_counts(image):
    s0c0 = image.select('Snow_Confidence').eq(0).And(image.select('Cloud_Confidence').eq(0))
    s1c0 = image.select('Snow_Confidence').eq(1).And(image.select('Cloud_Confidence').eq(0))
    s0c1 = image.select('Snow_Confidence').eq(0).And(image.select('Cloud_Confidence').eq(1))
    s1c1 = image.select('Snow_Confidence').eq(1).And(image.select('Cloud_Confidence').eq(1))

    counts = {
        's0c0': int(s0c0.reduceRegion(ee.Reducer.sum(), geometry, scale=30).get('Snow_Confidence').getInfo() or 0),
        's1c0': int(s1c0.reduceRegion(ee.Reducer.sum(), geometry, scale=30).get('Snow_Confidence').getInfo() or 0),
        's0c1': int(s0c1.reduceRegion(ee.Reducer.sum(), geometry, scale=30).get('Snow_Confidence').getInfo() or 0),
        's1c1': int(s1c1.reduceRegion(ee.Reducer.sum(), geometry, scale=30).get('Snow_Confidence').getInfo() or 0)
    }

    return counts

In [10]:
def process_monthly_data(year, month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')

    image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(start_date, end_date).filterDate(start_date, end_date).filterBounds(geometry).first()

    image_ndsi_conf = calculate_ndsi_and_confidence(image)

    pixel_counts = calculate_pixel_counts(image_ndsi_conf)

    return {
        'year': year,
        'month': month,
        's0c0': pixel_counts['s0c0'],
        's1c0': pixel_counts['s1c0'],
        's0c1': pixel_counts['s0c1'],
        's1c1': pixel_counts['s1c1']
    }

# Loop over the years and months to generate the dataframe
results = []
for year in range(2013, 2024):
    if year == 2013:
        for month in range(4,13):
            result = process_monthly_data(year, month)
            results.append(result)
    else:    
        for month in range(1, 13):
            result = process_monthly_data(year, month)
            results.append(result)

# Convert the results to a DataFrame
df = pd.DataFrame(results)

# Display the DataFrame
print(df)

     year  month    s0c0   s1c0    s0c1  s1c1
0    2013      4   50597  36094  157484     0
1    2013      5   51440  71532  120772     0
2    2013      6   35263  42887  166026     0
3    2013      7   44537  58378  141261     0
4    2013      8   14118   9153  220904     0
..    ...    ...     ...    ...     ...   ...
124  2023      8   49624  37174  157377     0
125  2023      9   82249    359  156412     0
126  2023     10  172963  29234   41979     0
127  2023     11   53284  44634  144278     0
128  2023     12    4484      0  239692     0

[129 rows x 6 columns]


In [14]:
def get_bands_data(year, month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(start_date, end_date).filterDate(start_date, end_date).filterBounds(geometry).first()
    image_ndsi_conf = calculate_ndsi_and_confidence(image)

    data = image_ndsi_conf.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=30, maxPixels=1e8).getInfo()

    ndsi_values = data.get('NDSI', [])
    snow_confidence_values = data.get('Snow_Confidence', [])
    cloud_confidence_values = data.get('Cloud_Confidence', [])
    swir = data.get('SWIR', [])
    green = data.get('Green', [])

    min_length = min(len(ndsi_values), len(swir), len(green), len(snow_confidence_values), len(cloud_confidence_values))
    
    if min_length == 0:
        print(f"No data for year {year}, month {month}. Skipping.")
        return []

    month_data = []
    for i in range(min_length):
        month_data.append({
            'year': year,
            'month': month,
            'ndsi': ndsi_values[i],
            'swir': swir[i],
            'green': green[i],
            's1c0': int(snow_confidence_values[i] == 1 and cloud_confidence_values[i] == 0),
            's0c1': int(snow_confidence_values[i] == 0 and cloud_confidence_values[i] == 1)
        })

    return month_data

results = []
for month in tqdm(range(1, 13)):
    month_data = get_bands_data(2014, month)
    results.extend(month_data)

df_bands = pd.DataFrame(results)

print(df_bands)

100%|██████████| 12/12 [02:29<00:00, 12.42s/it]


         year  month      ndsi   swir  green  s1c0  s0c1
0        2014      1  0.546370  10925  37242     1     0
1        2014      1  0.485096   8810  25410     1     0
2        2014      1  0.550795   9467  32683     1     0
3        2014      1  0.571165   9979  36561     1     0
4        2014      1  0.551812   9578  33163     1     0
...       ...    ...       ...    ...    ...   ...   ...
2929442  2014     12 -0.269306  10475  13261     0     0
2929443  2014     12 -0.276912  12474  12598     0     0
2929444  2014     12 -0.220872  13666  15103     0     0
2929445  2014     12 -0.292963  13900  12791     0     0
2929446  2014     12 -0.272054  12956  13471     0     0

[2929447 rows x 7 columns]


In [15]:
df_bands['swir'] = df_bands['swir']*0.0000275 - 0.2
df_bands['green'] = df_bands['green']*0.0000275 - 0.2